# 词的相似性和类比任务
:label:`sec_synonyms`

在 :numref:`sec_word2vec_pretraining`中，我们在一个小的数据集上训练了一个word2vec模型，并使用它为一个输入词寻找语义相似的词。实际上，在大型语料库上预先训练的词向量可以应用于下游的自然语言处理任务，这将在后面的 :numref:`chap_nlp_app`中讨论。为了直观地演示大型语料库中预训练词向量的语义，让我们将预训练词向量应用到词的相似性和类比任务中。


In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

## 加载预训练词向量

以下列出维度为50、100和300的预训练GloVe嵌入，可从[GloVe网站](https://nlp.stanford.edu/projects/glove/)下载。预训练的fastText嵌入有多种语言。这里我们使用可以从[fastText网站](https://fasttext.cc/)下载300维度的英文版本（“wiki.en”）。


In [2]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

为了加载这些预训练的GloVe和fastText嵌入，我们定义了以下`TokenEmbedding`类。


In [20]:
#@save
class TokenEmbedding:
    """GloVe嵌入"""
    # 构造函数
    def __init__(self, embedding_name):
        self.unknown_idx = 0
        self.idx_to_token, self.idx_to_vec = self._load_embedding(embedding_name)
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)} # 这是一个字典，将词（token）映射到索引（idx）

    # 私有方法
    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            # 每行格式：word v1 v2 v3 ... vn
            # 第一个字段是词（token），其余是浮点数形式的向量分量
            for line in f:
                elems = line.rstrip().split(' ')
                # token是词（token）
                # elems是浮点数形式的向量
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过标题信息，例如fastText中的首行
                if len(elems) > 1:
                    idx_to_token.append(token) # 一维，每个元素是一个词（token）
                    idx_to_vec.append(elems) # 二维，每个元素是一个向量（elems）
        # 为 <unk> 构造一个零向量，借助idx_to_vec[0]的维度，因为所有向量的维度都相同
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)
        # idx_to_token：一维，每个元素是一个词（token）
        # idx_to_vec：二维，每个元素是一个向量（elems）

    # 公开方法
    # 给定一个词列表 tokens，返回对应的词向量张量
    def __getitem__(self, tokens):
        # 若不存在，返回 unknown_idx（即 0）
        indices = [self.token_to_idx.get(token, self.unknown_idx) for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)] # 支持批量索引
        # vecs(len(tokens), 向量维度E)
        return vecs
    
    # 公开方法
    def __len__(self):
        return len(self.idx_to_token)

下面我们加载50维GloVe嵌入（在维基百科的子集上预训练）。创建`TokenEmbedding`实例时，如果尚未下载指定的嵌入文件，则必须下载该文件。


In [21]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

输出词表大小。词表包含400000个词（词元）和一个特殊的未知词元。


In [22]:
len(glove_6b50d)

400001

我们可以得到词表中一个单词的索引，反之亦然。


In [25]:
print(glove_6b50d.token_to_idx['beautiful'])
print(glove_6b50d.idx_to_token[3367])
print(glove_6b50d.idx_to_vec[3367])
print(glove_6b50d[['beautiful']])
print(glove_6b50d[['beautiful']].shape)


3367
beautiful
tensor([ 0.5462,  1.2042, -1.1288, -0.1325,  0.9553,  0.0405, -0.4786, -0.3397,
        -0.2806,  0.7176, -0.5369, -0.0046,  0.7322,  0.1210,  0.2809, -0.0881,
         0.5973,  0.5526,  0.0566, -0.5025, -0.6320,  1.1439, -0.3105,  0.1263,
         1.3155, -0.5244, -1.5041,  1.1580,  0.6880, -0.8505,  2.3236, -0.4179,
         0.4452, -0.0192,  0.2897,  0.5326, -0.0230,  0.5896, -0.7240, -0.8522,
        -0.1776,  0.1443,  0.4066, -0.5200,  0.0908,  0.0830, -0.0220, -1.6214,
         0.3458, -0.0109])
tensor([[ 0.5462,  1.2042, -1.1288, -0.1325,  0.9553,  0.0405, -0.4786, -0.3397,
         -0.2806,  0.7176, -0.5369, -0.0046,  0.7322,  0.1210,  0.2809, -0.0881,
          0.5973,  0.5526,  0.0566, -0.5025, -0.6320,  1.1439, -0.3105,  0.1263,
          1.3155, -0.5244, -1.5041,  1.1580,  0.6880, -0.8505,  2.3236, -0.4179,
          0.4452, -0.0192,  0.2897,  0.5326, -0.0230,  0.5896, -0.7240, -0.8522,
         -0.1776,  0.1443,  0.4066, -0.5200,  0.0908,  0.0830, -0.0220, -

## 应用预训练词向量

使用加载的GloVe向量，我们将通过下面的词相似性和类比任务中来展示词向量的语义。

### 词相似度

与 :numref:`subsec_apply-word-embed`类似，为了根据词向量之间的余弦相似性为输入词查找语义相似的词，我们实现了以下`knn`（$k$近邻）函数。


In [ ]:
# W: 词向量矩阵 (V, E)
# x: 查询向量 (E,)
# k: 返回前k个最相似的向量索引
def knn(W, x, k):
    # 增加1e-9以获得数值稳定性
    cos = torch.mv(W, x.reshape(-1,)) / (torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) * torch.sqrt((x * x).sum()))
    # 返回:前k个相似度值、前k个索引
    similarity_values, topk = torch.topk(cos, k=k)
    return topk, similarity_values.tolist()

然后，我们使用`TokenEmbedding`的实例`embed`中预训练好的词向量来搜索相似的词。


In [83]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # 排除输入词
        print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')

`glove_6b50d`中预训练词向量的词表包含400000个词和一个特殊的未知词元。排除输入词和未知词元后，我们在词表中找到与“chip”一词语义最相似的三个词。


In [84]:
get_similar_tokens('intel', 10, glove_6b50d)


ibm：cosine相似度=0.881
amd：cosine相似度=0.872
motorola：cosine相似度=0.852
cisco：cosine相似度=0.838
microsoft：cosine相似度=0.827
compaq：cosine相似度=0.798
packard：cosine相似度=0.783
hewlett：cosine相似度=0.779
microprocessor：cosine相似度=0.763
pentium：cosine相似度=0.754


下面输出与“baby”和“beautiful”相似的词。


In [75]:
get_similar_tokens('baby', 3, glove_6b50d)

babies：cosine相似度=0.839
boy：cosine相似度=0.800
girl：cosine相似度=0.792


In [78]:
get_similar_tokens('beautiful', 3, glove_6b50d)

lovely：cosine相似度=0.921
gorgeous：cosine相似度=0.893
wonderful：cosine相似度=0.830


### 词类比

除了找到相似的词，我们还可以将词向量应用到词类比任务中。
例如，“man” : “woman” :: “son” : “daughter”是一个词的类比。
“man”是对“woman”的类比，“son”是对“daughter”的类比。
具体来说，词类比任务可以定义为：
对于单词类比$a : b :: c : d$，给出前三个词$a$、$b$和$c$，找到$d$。
用$\text{vec}(w)$表示词$w$的向量，
为了完成这个类比，我们将找到一个词，
其向量与$\text{vec}(c)+\text{vec}(b)-\text{vec}(a)$的结果最相似。


In [124]:
"""
    根据词向量进行类比推理，返回与类比结果最相似的词。

    参数
    ----------
    token_a : str
        类比关系中的第一个词，例如 "man"。
    token_b : str
        类比关系中的第二个词，例如 "woman"。
    token_c : str
        类比关系中的第三个词，例如 "king"。
    embed : gluonnlp.embedding.TokenEmbedding
        预训练好的词向量模型，用于查找词的向量表示。

    返回
    -------
    str
        与类比关系 "token_a 对应 token_b，那么 token_c 对应 ?" 最相似的词。
"""
def get_analogy(token_a, token_b, token_c, embed):
     # 获取 token_a, token_b, token_c 的向量表示
    vecs = embed[[token_a, token_b, token_c]]
    # 计算 vecs[1] - vecs[0] + vecs[2] 的向量表示
    x = vecs[1] - vecs[0] + vecs[2]
    # 找到与 x 最相似的向量表示
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])], cos[0]
    # 没有排除第一个词, 所以返回的词可能是 token_d 本身(cos==1)

让我们使用加载的词向量来验证“male-female”类比。


In [125]:
get_analogy('man', 'woman', 'son', glove_6b50d)

('daughter', 0.9658340811729431)

下面完成一个“首都-国家”的类比：
“beijing” : “china” :: “tokyo” : “japan”。
这说明了预训练词向量中的语义。


In [128]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)


('japan', 0.9054064750671387)

另外，对于“bad” : “worst” :: “big” : “biggest”等“形容词-形容词最高级”的比喻，预训练词向量可以捕捉到句法信息。


In [130]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

('biggest', 0.8059625625610352)

为了演示在预训练词向量中捕捉到的过去式概念，我们可以使用“现在式-过去式”的类比来测试句法：“do” : “did” :: “go” : “went”。


In [131]:
get_analogy('do', 'did', 'go', glove_6b50d)

('went', 0.924229621887207)

## 小结

* 在实践中，在大型语料库上预先练的词向量可以应用于下游的自然语言处理任务。
* 预训练的词向量可以应用于词的相似性和类比任务。

## 练习

1. 使用`TokenEmbedding('wiki.en')`测试fastText结果。
1. 当词表非常大时，我们怎样才能更快地找到相似的词或完成一个词的类比呢？


[Discussions](https://discuss.d2l.ai/t/5746)
